<a href="https://colab.research.google.com/github/akshat3011/punchline-detection-hierarchical-fusion/blob/master/punchline_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#path in drive to UR-FUNNY data
%cd gdrive/My\ Drive/HF-Network/UR-FUNNY

/content/gdrive/My Drive/HF-Network/UR-FUNNY


In [3]:
import humor_dataloader as hd
import pickle 
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [4]:
# Loading data
data_folds_file= "data_folds.pkl"
data_folds= hd.load_pickle(data_folds_file)
train=data_folds['train']
dev=data_folds['dev']
test=data_folds['test']

In [5]:
len(train)+len(dev)+len(test)

16514

In [6]:
output_dir = "/content/gdrive/My Drive/HF-Network/output/"
weights_dir = "/content/gdrive/My Drive/HF-Network/pretrained_weights/"

In [7]:
data_len = {'test': len(test),'train': len(train)}

In [8]:
train_set = hd.HumorDataset(train)

In [9]:
dev_set = hd.HumorDataset(dev)

In [10]:
test_set = hd.HumorDataset(test)

In [11]:
batch = 10
train_dataloader = hd.DataLoader(train_set, batch_size=batch, shuffle=True)
dev_dataloader = hd.DataLoader(dev_set, batch_size=batch, shuffle=True)
test_dataloader = hd.DataLoader(test_set, batch_size=batch, shuffle=True)

In [12]:
# dimension equilization dimension
dim_proj = 400 
text_dim, audio_dim,video_dim = 300, 81, 75
dim = audio_dim + video_dim + text_dim
max_context_len = 5
max_sen_len = 20

In [13]:
def crop(a):
    return a[:,:,:, 0:text_dim],a[:, :, :, text_dim:(text_dim + audio_dim)], a[:, :, :, (text_dim+audio_dim):dim]

In [14]:
def crop_1(a):
    return a[:,:, 0:text_dim],a[:,:, text_dim:(text_dim + audio_dim)],a[:,:,(text_dim + audio_dim):dim]

In [15]:
class hadamard(nn.Module):
    def __init__(self):
        super(hadamard, self).__init__()
        self.w1 = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))
        self.w2 = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))
        self.bias = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))

    def forward(self, x):
        return x[0] * self.w1 + x[1] * self.w2 + self.bias

In [16]:
#For dimension equalization of text modality 
class fc1(nn.Module):
    def __init__(self):
        super(fc1, self).__init__()
        self.fc = nn.Linear(text_dim, dim_proj)
    def forward(self,x):
        return self.fc(x)

#For dimension equalization of audio modality 
class fc2(nn.Module):
    def __init__(self):
        super(fc2, self).__init__()
        self.fc = nn.Linear(audio_dim, dim_proj)
    def forward(self,x):
        return self.fc(x)

#For dimension equalization of video modality 
class fc3(nn.Module):
    def __init__(self):
        super(fc3, self).__init__()
        self.fc = nn.Linear(video_dim, dim_proj)
    def forward(self,x):
        return self.fc(x)

In [17]:
# Bimodal Fusion for text and audio
class bimodal_1_2(nn.Module):
    def __init__(self):
        super(bimodal_1_2, self).__init__()
        self.fc1 = fc1()
        self.fc2 = fc2()
        self.act = nn.Tanh()
        self.hadamard = hadamard()
        self.fcp = nn.Linear(max_sen_len*dim_proj, 2)
        self.dropout = nn.Dropout(p = 0.5)
    
    def forward(self,out1,out2,out3):
        bs = out1.shape[0]
        out1 = self.fc1(out1)
        out2 = self.fc2(out2)
        out1 = self.act(out1)
        out2 = self.act(out2)
        out1  = self.dropout(out1)
        out2 = self.dropout(out2)
        out  = self.hadamard([out1,out2])
        out = self.act(out)
        out_f = out.view(bs,6,-1)
        out_f = self.fcp(out_f)
        return out_f, out

In [18]:
# Bimodal Fusion for audio and video
class bimodal_2_3(nn.Module):
    def __init__(self):
        super(bimodal_2_3, self).__init__()
        self.fc2 = fc2()
        self.fc3 = fc3()
        self.act = nn.Tanh()
        self.hadamard = hadamard()
        self.crop = crop
        self.fcp = nn.Linear(max_sen_len*dim_proj, 2)
        self.dropout = nn.Dropout(p = 0.5)
    
    def forward(self,out1,out2,out3):
        bs = out2.shape[0]
        out2 = self.fc2(out2)
        out3 = self.fc3(out3)
        out2 = self.act(out2)
        out3 = self.act(out3)
        out2 = self.dropout(out2)
        out3 = self.dropout(out3)
        out  = self.hadamard([out2,out3])
        out = self.act(out)
        out_f = out.view(bs,6,-1)
        out_f = self.fcp(out_f)
        return out_f,out

In [19]:
# Bimodal Fusion for text and video
class bimodal_1_3(nn.Module):
    def __init__(self):
        super(bimodal_1_3, self).__init__()
        self.fc1 = fc1()
        self.fc3 = fc3()
        self.act = nn.Tanh()
        self.hadamard = hadamard()
        self.crop = crop
        self.fcp = nn.Linear(max_sen_len*dim_proj, 2)
        self.dropout = nn.Dropout(p = 0.5)
    
    def forward(self,out1,out2,out3): #dim of each is batch_size*110*100
        bs = out1.shape[0]
        out1 = self.fc1(out1)
        out3 = self.fc3(out3)
        out1 = self.act(out1)
        out3 = self.act(out3)
        out1 = self.dropout(out1)
        out3 = self.dropout(out3)
        out  = self.hadamard([out1,out3])
        out = self.act(out)
        out_f = out.view(bs,6,-1)
        out_f = self.fcp(out_f)
        return out_f,out

In [20]:
data_loaders = {"train": train_dataloader, "test": test_dataloader}

In [21]:
net1 = bimodal_1_2()
net2 = bimodal_2_3()
net3 = bimodal_1_3()
num_epochs = 150

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(net1.parameters(), lr = 0.0001)
optimizer2 = optim.Adam(net2.parameters(), lr = 0.0001)
optimizer3 = optim.Adam(net3.parameters(), lr = 0.0001)

In [23]:
#function to train bimodal components
def train_bimodal(model,model_name,optimizer):

  running_loss = 0
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)

  print("Training Network: " + model_name)
  for epoch in range(num_epochs):
    print("Epoch :" + str(epoch+1))
    print("**********************************")
    for phase in ['train','test']:
      comb_y = []
      comb_o  = []

      if phase == 'train':
        model.train(True)  # Set model to training mode
      else:
        model.train(False)  # Set model to evaluate mode

      for  batch_idx, batch in enumerate(data_loaders[phase], 0):
        x_p,x_c,y = map(lambda x: x.to(device), batch)
        x_p = torch.unsqueeze(x_p, dim=1)
        combined = torch.cat([x_c, x_p], dim=1)
        # get vectors for each modality  
        x1,x2,x3 = crop(combined)
        optimizer.zero_grad()
        output,_ = model(x1,x2,x3)
        bs = x1.shape[0]
        output = output.to(device)
        op = torch.zeros(0).to(device)
        count = 0
        output = output.view(-1,2)

        for t in output:
          if(count%6==5):
            op = torch.cat([op,t],dim = 0)
          count = count + 1

        op = op.view(-1,2)
        op_sigm = torch.sigmoid(op)
        y = y.to(device=device, dtype=torch.int64)
        y = torch.squeeze(y, -1)

        # for calculating accuracy after each epoch
        for o in y:
          comb_y.append(o.item())
        for p in op_sigm:
          if(p[0].item() > p[1].item()):
            comb_o.append(0)
          else:
            comb_o.append(1)

        loss = criterion(op,y)
        running_loss += loss.item()

        if(phase == 'train'):
          loss.backward()
          optimizer.step()

      print(phase + ' loss: ' + str(running_loss/data_len[phase]))
      running_loss = 0.0
      print( phase + ' accuracy: ' + str(accuracy_score(comb_y,comb_o)))
    if((epoch+1)%5 == 0):
      print("Saving Model")
      torch.save(model.state_dict(), output_dir + "%s_%d.pth" % (model_name,epoch+1))

Net1 text + acoustic

In [ ]:
train_bimodal(net1,"net1",optimizer1)  

NET 2 acoustic + visual

In [ ]:
train_bimodal(net2,"net2",optimizer2) 

NET 3 text  + visual


In [ ]:
train_bimodal(net3,"net3",optimizer3) 

Loading best bimodal models for trimodal fusion


In [24]:
net3.load_state_dict(torch.load(weights_dir + "net3.pth"))

<All keys matched successfully>

In [25]:
net1.load_state_dict(torch.load(weights_dir + "net1.pth"))


<All keys matched successfully>

In [26]:
net2.load_state_dict(torch.load(weights_dir + "net2.pth"))

<All keys matched successfully>

TRIMODAL FUSION

In [27]:
gru_dim = 200
pro_dim = 900

In [28]:
class tri_hadamard(nn.Module):
    def __init__(self):
        super(tri_hadamard, self).__init__()
        self.w1 = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))
        self.w2 = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))
        self.w3 = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))
        self.bias = nn.Parameter(torch.zeros(6,max_sen_len,dim_proj))

    def forward(self, x):
        return x[0] * self.w1 + x[1] * self.w2 + x[2] * self.w3 + self.bias

In [29]:
#Context Modelling Network
class tri_GRU(nn.Module):
  def __init__(self):
      super(tri_GRU, self).__init__()
      self.input_dim = max_sen_len*dim_proj
      self.hidden_dim = gru_dim
      self.drop_prob = 0.4
      self.gru = nn.GRU(self.input_dim, self.hidden_dim, dropout= self.drop_prob,num_layers = 1)
      self.dropout = nn.Dropout(p = 0.5)
      self.act = nn.Tanh()
      self.fcp = nn.Linear(max_sen_len*dim_proj,pro_dim )

  def forward(self,x):
      x_p = x[:,1,:]
      x_c = x[:,1:6,:]
      x = self.gru(x_c)
      x = x[0]
      x = x[:,4,:]
      x = self.act(x)
      x_p = self.fcp(x_p)
      x = torch.cat([x,x_p],dim = 1)
      return x

In [30]:
class trimodal(nn.Module):
    def __init__(self):
        super(trimodal, self).__init__()
        self.hadamard = tri_hadamard()
        self.act = nn.Tanh()
        self.f1 = nn.Linear(max_sen_len*dim_proj, 450)
        self.fcp = nn.Linear(gru_dim+pro_dim, 2)
        self.dropout = nn.Dropout(p = 0.3)
        self.context_modelling = tri_GRU()
        self.crop = crop
        self.net1 = net1
        self.net2 = net2
        self.net3 = net3
    
    def forward(self,x1,x2,x3):
        _,out1 = self.net1(x1,x2,x3)
        _,out2 = self.net2(x1,x2,x3)
        _,out3 = self.net3(x1,x2,x3)
        bs = out2.shape[0]
        out1 = self.dropout(out1)
        out2 = self.dropout(out2)
        out3 = self.dropout(out3)
        out = self.hadamard([out1,out2,out3])
        out = self.act(out)
        out = out.view(bs,6,-1)
        out = self.context_modelling(out)
        out = self.fcp(out) 
        return out

In [31]:
trimodal()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


trimodal(
  (hadamard): tri_hadamard()
  (act): Tanh()
  (f1): Linear(in_features=8000, out_features=450, bias=True)
  (fcp): Linear(in_features=1100, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (context_modelling): tri_GRU(
    (gru): GRU(8000, 200, dropout=0.4)
    (dropout): Dropout(p=0.5, inplace=False)
    (act): Tanh()
    (fcp): Linear(in_features=8000, out_features=900, bias=True)
  )
  (net1): bimodal_1_2(
    (fc1): fc1(
      (fc): Linear(in_features=300, out_features=400, bias=True)
    )
    (fc2): fc2(
      (fc): Linear(in_features=81, out_features=400, bias=True)
    )
    (act): Tanh()
    (hadamard): hadamard()
    (fcp): Linear(in_features=8000, out_features=2, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (net2): bimodal_2_3(
    (fc2): fc2(
      (fc): Linear(in_features=81, out_features=400, bias=True)
    )
    (fc3): fc3(
      (fc): Linear(in_features=75, out_features=400, bias=True)
    )
    (act): Tanh()
    (hadam

In [32]:
tri_net = trimodal()
num_epochs = 150
batch=10

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [33]:
criterion = nn.CrossEntropyLoss()
tri_optimizer = optim.Adam(tri_net.parameters(), lr=0.00001)

In [34]:
tri_net.load_state_dict(torch.load(weights_dir + 'tri_net.pth'))

<All keys matched successfully>

In [35]:
#Training Trimodal component
running_loss = 0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tri_net = tri_net.to(device)
for epoch in range(num_epochs):

  print("Epoch :" + str(epoch+1))
  print("**********************************")
  for phase in ['train','test']:
    comb_y = []
    comb_o  = []
    if phase == 'train':
      tri_net.train(True)  # Set model to training mode
    else:
      tri_net.train(False)  # Set model to evaluate mode

    for  batch_idx, batch in enumerate(data_loaders[phase], 0):
      x_p,x_c,y = map(lambda x: x.to(device), batch)
      x_p = torch.unsqueeze(x_p, dim=1)
      combined = torch.cat([x_c, x_p], dim=1)
      x1,x2,x3 = crop(combined)

      tri_optimizer.zero_grad()
      output = tri_net(x1,x2,x3)
      bs = x1.shape[0]
      
      output = output.to(device)
      op_sigm = torch.sigmoid(output)
      y = y.to(device=device, dtype=torch.int64)
      output = output.view(-1,2)
    
      y = torch.squeeze(y, -1)
      for o in y:
        comb_y.append(o.item())
      for p in op_sigm:
        if(p[0].item()>p[1].item()):
          comb_o.append(0)
        else:
          comb_o.append(1)     
        
      loss = criterion(output,y)
      if(phase == 'train'):
        loss.backward()
        tri_optimizer.step()
      running_loss += loss.item()

    print(phase + ' loss: ' + str(running_loss/data_len[phase]))
    running_loss = 0.0
    print( phase + ' accuracy: ' + str(accuracy_score(comb_y,comb_o)))
  if((epoch+1)%1 == 0):
    print("Saving Model")
    torch.save(tri_net.state_dict(), output_dir + 'trimodal_%d.pth' % (epoch+1))